In [1]:
import matplotlib.pyplot as plt
import numpy as np
import viper_detector
from hcipy import *
from scipy.ndimage import gaussian_filter
import time
from utils import *
import copy
from hcipy import *
import os.path
import csv
from astropy.io import fits
from processing import *

# User Inputs

In [13]:
## Metadata
name = "Prime_BSI_Mag_5_Band_V" # filename you want to write, no extension
overwrite = False # True if you wish to overwrite files with the same name
detector_name = "Prime_BSI" 

## Properties for EMCCD
EM_gain = None # Set EM Gain for EMCCDs. If running a detector with no EM Gain, set = None
EM_saturation = None # Set behavior when full well depth is reached. None means saturated pixels will be automatically set to full well depth. np.nan means saturated pixels will be set to np.nan

## Properties of the Focal Grid
q=2 # Number of pixels per resolution element
nairy = 200 #The spatial extent of the grid radius in resolution elements (=lambda f/D)

## Exposure time and total number of exposures
exposure_time = 1./800 # Exposure time in seconds. For AO systems make this the frequency of the AO system.
exposure_total = 100 # Total number of exposures

## Setting Up the Atmosphere
seeing = 1.75
outer_scale = 40. # (meter) 
velocity = 30. # (m/s) 
                                    
## Setting up the telescope
pupil_diameter = 3.048 # (meter)
f_number = 13 # effective focal ratio
grid_size = 120 # Number of pixels per dimension
filter_name = 'V' # Name of filter
telescope_pupil_generator = make_lick_aperture()

## Add a Primary and Companion
# Primary parameters
mag = 5 # Magnitude in the band of interest
stokes_vector= [1.,0.,0.,0.] # I, Q, U, V
# Companion parameters
contrast = 0.
stokes_ps = [1.,0.,0.,0.] # I, Q, U, V
angular_separation= 2 # Lambda/D

## AO Parameters
#leaky integrator parameters
gain = 0.3
leakage = 0.999

#AO loop speed: = Exposure Time
dt= exposure_time

num_actuators = 11 # set the number of actuators

# setup Pyramid WFS
pixels_pyramid_pupils=20 # number of pixels across the pupil; want 120 %(mod) pixels_pyramid_pupils =0

mld=5 # modulation radius in lambda/D 

modsteps = 12 #keep this as a factor of 4. Significantly increases computation time.


# Helper Functions

The following defines some some important helper functions. Please run this section as well. It is hidden to declutter the notebook for you.

bin(imin,fbin) This binning function takes an image of a certain size and bins it down by a certain amount.

pyramid_slopes (image) This function calculates the slopes for our pyramid wavefront sensor.

plot_slopes (image) This function helps to make data that is easy to plot.

In [3]:
#@title
 #helper functions 

def bin(imin,fbin): 

    ''' Parameters
    ----------
    imin : 2D numpy array
         The 2D image that you want to bin
    fbin : int
         
    
    Returns
    -------
    out : 2D numpy array
        the 2D binned image
        '''
    out=np.zeros((int(imin.shape[0]/fbin),int(imin.shape[1]/fbin)))
   #  begin binning
    for i in np.arange(fbin-1,imin.shape[0]-fbin,fbin):
        for j in np.arange(fbin-1,imin.shape[1]-fbin,fbin):
            out[int((i+1)/fbin)-1,int((j+1)/fbin)-1]=np.sum(imin[i-int((fbin-1)/2):i+int((fbin-1)/2),j-int((fbin-1)/2):j+int((fbin-1)/2)])
    return out


def pyramid_slopes(image,pixels_pyramid_pupils):

    ''' Parameters
    ----------
    image : 1D numpy array
         The flatted image of the pyramid wfs pupils
           
    Returns
    -------
    slopes : 1D numpy array
        x- and y- slopes inside the pupil stacked onto of eachother for 1D array
        '''
    D=4 #hardcoded for now/ease
    pyramid_plot_grid = make_pupil_grid(pixels_pyramid_pupils*2, D) #hardcoded for now/ease

    pyr1=circular_aperture(0.5*D,[-0.25*D,0.25*D])(pyramid_plot_grid)
    pyr2=circular_aperture(0.5*D,[0.25*D,0.25*D])(pyramid_plot_grid)
    pyr3=circular_aperture(0.5*D,[-0.25*D,-0.25*D])(pyramid_plot_grid)
    pyr4=circular_aperture(0.5*D,[0.25*D,-0.25*D])(pyramid_plot_grid)
    N=4*np.sum(pyr1[pyr1>0])
    norm=(image[pyr1>0]+image[pyr2>0]+image[pyr3>0]+image[pyr4>0])/N
    sx=(image[pyr1>0]-image[pyr2>0]+image[pyr3>0]-image[pyr4>0])
    sy=(image[pyr1>0]+image[pyr2>0]-image[pyr3>0]-image[pyr4>0])
    return np.array([sx,sy]).flatten()

def plot_slopes(slopes,pixels_pyramid_pupils):
    ''' 
    Only want if we decide to plot the slopes. 

    Parameters
    ----------
    slopes : 1D numpy array
         The flatted slopes produced by pyramid_slopes(). 
           
    Returns
    -------
    slopes : 1D numpy array
        x- and y- slopes mapped within their pupils for easy plotting
    '''
    D=4
    mid=int(slopes.shape[0]/2)
    pyramid_plot_grid = make_pupil_grid(pixels_pyramid_pupils, D) #hardcoded for now/ease
    pyr_mask=circular_aperture(D)(pyramid_plot_grid)
    sx=pyr_mask.copy()
    sy=pyr_mask.copy()
    sx[sx>0]=slopes[0:mid]
    sy[sy>0]=slopes[mid::]
    return [sx,sy]



The hidden cell below contains the function make_command_matrix().

In [4]:
#@title

def make_command_matrix(deformable_mirror, mpwfs,modsteps,wfs_camera,wf,pixels_pyramid_pupils):

  probe_amp = 0.02 * wf.wavelength
  response_matrix = []
  num_modes=deformable_mirror.num_actuators

  for i in range(int(num_modes)):
      slope = 0

      for s in [1, -1]:
          amp = np.zeros((num_modes,))
          amp[i] = s * probe_amp
          deformable_mirror.flatten()
          deformable_mirror.actuators = amp

          dm_wf = deformable_mirror.forward(wf)
          wfs_wf = mpwfs.forward(dm_wf)

          for m in range (modsteps) :
                wfs_camera.integrate(wfs_wf[m], 1)
                
          image_nophot = bin(wfs_camera.read_out().shaped,pyr_bin).flatten()
          image_nophot/=image_nophot.sum()
          sxy=pyramid_slopes(image_nophot,pixels_pyramid_pupils)

          slope += s * (sxy-pyr_ref)/(2*probe_amp)  #these are not really slopes; this is just a normalized differential image

      response_matrix.append(slope.ravel())
    
  response_mtx= ModeBasis(response_matrix)
  rcond = 1e-3

  reconstruction_matrix = inverse_tikhonov(response_mtx.transformation_matrix, rcond=rcond)

  return reconstruction_matrix

# Generating the FITS File, Atmosphere, Focal Grid, Detector

In [19]:
## Some math to define additional parameters
collecting_area = np.pi * (3.048**2 - 0.9779**2)
effective_focal_length = pupil_diameter * f_number # (meter)
wavelength = filters[filter_name]['lambda_eff'] * 1e-6 # (meter)

## Generate name strings
fits_name = name + ".fits"

## Checking to see if filenames exists
if os.path.isfile(fits_name):
    if overwrite:
        print("File name ", fits_name, " already exists. Preparing to overwrite.") 
        os.remove(fits_name)
    else:
        print("Error, file name ",fits_name," already exists. Overwrite was not allowed.")
        print("Exiting program...")
        quit()

## Generating the pupil grid
print("Generating the pupil grid")
pupil_grid = make_pupil_grid(grid_size, diameter=pupil_diameter)

## Adjust spiders to represent Shane pupil
print("Generating the telescope pupil")
telescope_pupil = telescope_pupil_generator(pupil_grid)

## Generating the atmosphere
print("Generating the atmosphere")
fried_parameter = seeing_to_fried_parameter(seeing, wavelength)                             
Cn_squared = Cn_squared_from_fried_parameter(fried_parameter, wavelength)
tau0 = 0.314 * fried_parameter/velocity
layer = InfiniteAtmosphericLayer(pupil_grid, Cn_squared, outer_scale, velocity)

##Generating the focal grid
print("Generating the focal grid")
focal_grid = make_focal_grid(q=q, 
                             num_airy=nairy,
                             pupil_diameter=pupil_diameter,
                             focal_length = effective_focal_length,
                             reference_wavelength=wavelength)

## Define the Detector
call_detector = "viper_detector." + detector_name + "(focal_grid, " + f'"{filter_name}"'
if EM_gain == None:
    call_detector += ")"
else:
    call_detector += ", " + str(EM_gain)+ ", " + str(EM_saturation) +")"
print(call_detector)
detector = eval(call_detector)                        

## Generating the propagator
print("Generating the propagator")
prop = FraunhoferPropagator(pupil_grid, focal_grid, 
                            focal_length=effective_focal_length)

## Generating wavefront of primary and companion
print("Generating wavefront of primary and companion")
pupil_wavefront = Wavefront(telescope_pupil, wavelength,
                            input_stokes_vector=stokes_vector)
pupil_wavefront.total_power = number_of_photons(mag,filter_name,collecting_area,)#In photons/s
wf_planet = Wavefront(telescope_pupil*np.exp(4j*np.pi*pupil_grid.x*angular_separation/pupil_diameter),
                      wavelength,
                      input_stokes_vector=stokes_ps)
wf_planet.total_power = contrast * number_of_photons(mag,filter_name,collecting_area,)# (photons/s)


Generating the pupil grid
Generating the telescope pupil
Generating the atmosphere
Generating the focal grid
viper_detector.Prime_BSI(pupil_grid, "V")
Generating the propagator
Generating wavefront of primary and companion


# Setting up the AO

In [21]:
#make the DM
actuator_spacing = pupil_diameter / num_actuators
influence_functions = make_gaussian_influence_functions(pupil_grid, num_actuators, actuator_spacing) 
deformable_mirror = DeformableMirror(influence_functions)

modradius = mld*wavelength/pupil_diameter # modulation radius in radians;
 
pwfs = PyramidWavefrontSensorOptics(pupil_grid, wavelength_0=wavelength, q=q, num_airy = nairy)
mpwfs = ModulatedPyramidWavefrontSensorOptics(pwfs,modradius,modsteps)   
wfs_camera = NoiselessDetector(pupil_grid)

#bin our pyramid image
pyramid_plot_grid = make_pupil_grid(pixels_pyramid_pupils*2, pupil_diameter)
pyr_bin=int((grid_size*2)/(2*pixels_pyramid_pupils))

#commands to modulate the PyWFS, get an image out, and calculate a reference slope
for m in range (modsteps) :
    
    wfs_camera.integrate(mpwfs(pupil_wavefront)[m], 1)
    wfs_camera.integrate(mpwfs(wf_planet)[m], 1)
pyr_ref = bin(wfs_camera.read_out().shaped,pyr_bin).flatten()
pyr_ref=pyramid_slopes(pyr_ref/pyr_ref.sum(),pixels_pyramid_pupils)

#Make command matrix for controller. 
#This code will have to be rerun everytime you change a parameter about the PyWFS or DM. 
#Just run this code & do not peak at the function that does the work for you. 
CM=make_command_matrix(deformable_mirror, mpwfs, modsteps,wfs_camera,wf,pixels_pyramid_pupils)


KeyboardInterrupt: 

In [ ]:
num_iterations = 400 #number of iterations in our simulation
sr=[] # so we can find the average strehl ratio

# create figure
fig=plt.figure(figsize=(15,8))

# generate animation object; two optional backends FFMpeg or GifWriter. 
anim = FFMpegWriter('AO_simulations_standard.mp4', framerate=3)
#anim = GifWriter('AO_simulations_standard.gif', framerate=3)
AO_res=[]
layer.t = 0
for timestep in range(num_iterations): 
    #get a clean wavefront
    wf_in=wf.copy()

    #evolve the atmospheric turbulence
    layer.t = timestep*dt
    
    #pass the wavefront through the turbulence
    wf_after_atmos = layer.forward((wf_in))

    #pass the wavefront through the DM for correction
    wf_after_dm = deformable_mirror.forward(wf_after_atmos)

    #send the wavefront containing the residual wavefront error to the PyWFS and get slopes
    wfs_wf = mpwfs.forward(wf_after_dm)
    for mmm in range (modsteps) :
              wfs_camera.integrate(wfs_wf[mmm], dt/modsteps)
    wfs_image = bin(wfs_camera.read_out().shaped,pyr_bin).flatten()    
    slopes = pyramid_slopes(wfs_image/wfs_image.sum(),pixels_pyramid_pupils) -pyr_ref 
    slopes = slopes.ravel()

    #res=layer.phase_for(wavelength)/k-deformable_mirror.opd #units:meters; use this to save to avoid phase wrapping that is in the electric field of wf
    res=wf_after_dm.phase/k
    AO_res.append(res)
    #Leaky integrator to calculate new DM commands    
    deformable_mirror.actuators =  leakage*deformable_mirror.actuators - gain * CM.dot(slopes)
  
    # Propagate to focal plane
    wf_focal = propagator.forward(wf_after_dm )
    
    #calculate the strehl ratio to use as a metric for how well the AO system is performing. 
    strehl_foc=get_strehl_from_focal(wf_focal.intensity/norm,im_ref.intensity/norm)
    sr.append(strehl_foc)
    #plot the results                
    if timestep % 25 == 0: #change this if you want to have more or less frames saved to the image. 
        plt.close(fig)
        fig=plt.figure(figsize=(15,8))
        plt.suptitle('Time %.2f s / %d s' % (timestep*dt, dt*num_iterations))

        plt.subplot(1,3,1)
        plt.title('WFS slopes')
        sxy=np.asarray(plot_slopes(slopes,pixels_pyramid_pupils)).reshape((2,pixels_pyramid_pupils,pixels_pyramid_pupils))
        plt.imshow(sxy.reshape((2*pixels_pyramid_pupils,pixels_pyramid_pupils)).transpose())
        plt.colorbar()

        plt.subplot(1,3,2)
        plt.title('Residual wavefront error [meters]')
        imshow_field(res*telescope_pupil, cmap='RdBu')
        plt.colorbar()
        
        plt.subplot(1,3,3)
        plt.title('Inst. PSF; Strehl %.2f'% (np.mean(np.asarray(sr))))
        imshow_field(np.log10(wf_focal.intensity/norm), cmap='inferno')
        plt.colorbar()
        plt.subplots_adjust(hspace=0.3)
        anim.add_frame()
#plt.suptitle('Gain = %.2f' % (gain)) # can change this to be the parameter you are varying
#plt.savefig('AO_vary_gain%.2f.png' % (gain)) #example to save the last figure to see how the parameter varied your performance
AO_res=np.asarray(AO_res)
np.save('AO_res_meters.npy',AO_res) 
plt.close()
anim.close()
anim

# plt.plot(np.array(sr))
# plt.show()